In [26]:
import os, sys, random, copy, json, time
import numpy as np
import cv2
import imutils
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Conv2D, Dropout, 
                                     Flatten, MaxPooling2D)
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../')

In [4]:
from utils.utils import makeMini, myPlot, answerToPoints, meanTruth, newVals, iterMean
from scorer_scripts_v2.scorer_scripts_v2.score_detections import score
from starter_scripts_v2.starter_scripts_v2.helper import plot_bbox

from utils.misc import contrivedLabel, plotLabel, summaryStats, bagData
from models.model import ModelBuilder
from models.data import DataXPrep, DataYPrep

In [5]:
p_truth = '../truth_new.json'
with open(p_truth, 'r') as f:
    truth = json.load(f)
list(truth.items())[:3]

[('IMG_9622.JPG', [[583, 439, 772, 431, 779, 628, 582, 631]]),
 ('IMG_9925.JPG', [[481, 295, 769, 268, 777, 590, 480, 589]]),
 ('IMG_6733.JPG', [[593, 362, 650, 399, 649, 597, 589, 592]])]

In [6]:
truth_0 = {}
for k in truth.keys():
    if len(truth[k][0]) > 0:
        truth_0[k] = truth[k][0][0]

truth_1 = {}
for k in truth.keys():
    if len(truth[k][0]) > 0:
        truth_1[k] = truth[k][0][1]
        
print (list(truth_0.items())[:3], '\n', list(truth_1.items())[:3])

[('IMG_9622.JPG', 583), ('IMG_9925.JPG', 481), ('IMG_6733.JPG', 593)] 
 [('IMG_9622.JPG', 439), ('IMG_9925.JPG', 295), ('IMG_6733.JPG', 362)]


In [7]:
p_training = '../Data_Training/Data_Training/'
def loadImg(fn):
    return cv2.imread(p_training + fn)

In [12]:
random.seed(a=0, version=2)
N = 1000
batch = random.sample(list(truth_0.keys()), N)  # use truth_0 with no missing rects

In [13]:
x_load = [loadImg(k) for k in batch]

y0_load = [truth_0[k] for k in batch]
y1_load = [truth_1[k] for k in batch]

In [22]:
from models.data import DataXPrep, DataYPrep

In [15]:
yPrep = DataYPrep()
y0 = yPrep.normData_static(y0_load)
y1 = yPrep.normData_static(y1_load)

In [16]:
xPrep = DataXPrep(data=x_load)
xPrep.buildDataT(
               b_resize=True,
               resize_width=108,
               b_cvt_grayscale=False
              )
x_t = xPrep.getDataT()
input_shape = xPrep.getImgTShape()

In [23]:
xPrep = DataXPrep(data=x_load)
xPrep.buildDataT(
               b_resize=True,
               resize_width=216,
               b_cvt_grayscale=False
              )
x_t_large = xPrep.getDataT()
input_shape_large = xPrep.getImgTShape()

In [24]:
input_shape_large

(144, 216, 3)

In [17]:
from tensorflow.keras.layers import (Dense, Conv2D, Dropout, 
                                     Flatten, MaxPooling2D)

### Run Param Loop:

results in mb_y0, mb_y1, time_log, time_log_y1

set EPOCHS = 2 or 100

In [38]:
#free up memory? doesn't
del x_load

In [39]:
mb_y0 = ModelBuilder()
mb_y1 = ModelBuilder()

In [ ]:
EPOCHS = 100
time_log = {}

for scenario in ['a','b','c','d','e','f','g','h','i','j','k',
                 'cd', 'fg','ec' ]:
    
    #defaults
    _vsplit = 0.5
    _epochs = EPOCHS
    _filters = 28
    _kerneln = 3
    _denseunits = 128
    _dropoutfrac = 0.2
    _lr = 0.001
    _inputshape = input_shape
    _x = x_t
    
    if 'a' in scenario:
        pass
    if 'b' in scenario:
        _epochs = EPOCHS*2
    if 'c' in scenario:
        _filters = 28*2
    if 'd' in scenario:
        _kerneln = 6
    if 'e' in scenario:
        _inputshape = input_shape_large
        _x = x_t_large
    if 'f' in scenario:
        _lr=0.002
    if 'g' in scenario:
        _denseunits = 256
    if 'h' in scenario:
        _dropoutfrac=0.3
    if 'i' in scenario:
        _dropoutfrac=0.1
    if 'j' in scenario:
        v_split = 0.2

    print('-------------------------------------------------------')
    print(scenario)
    
    mb_y0.setInputShape(_inputshape)
    
    mb_y0.buildModel(filters=_filters,
                     kernel_n=_kerneln,
                     dense_units=_denseunits,
                     dropout_frac=_dropoutfrac,
                     lr=_lr
                     
                    )
    
    t0 = time.time()
    
    mb_y0.fitModel(x=_x, y=y0, epochs=_epochs, validation_split=_vsplit)
    
    t1 = time.time()
    
    time_log[scenario] = t1 - t0
    
    


-------------------------------------------------------
a
Train on 500 samples, validate on 500 samples
Epoch 1/100
500/500 [==============================] - 6s 12ms/sample - loss: 32831551.7368 - mean_absolute_error: 1722.4116 - mean_squared_error: 32831554.0000 - val_loss: 15204.8788 - val_mean_absolute_error: 95.8133 - val_mean_squared_error: 15204.8779
Epoch 2/100
500/500 [==============================] - 5s 11ms/sample - loss: 52143.1304 - mean_absolute_error: 181.3826 - mean_squared_error: 52143.1367 - val_loss: 46983.2733 - val_mean_absolute_error: 198.5157 - val_mean_squared_error: 46983.2695
Epoch 3/100
500/500 [==============================] - 5s 10ms/sample - loss: 60869.0177 - mean_absolute_error: 196.9146 - mean_squared_error: 60869.0156 - val_loss: 12960.3854 - val_mean_absolute_error: 92.7771 - val_mean_squared_error: 12960.3838
Epoch 4/100
500/500 [==============================] - 5s 10ms/sample - loss: 49224.1742 - mean_absolute_error: 179.5175 - mean_squared_error

In [ ]:
EPOCHS = 100
time_log_y1 = {}

for scenario in ['a','b','c','d','e','f','g','h','i','j','k',
                 'cd', 'fg','ec' ]:
    
    #defaults
    _vsplit = 0.5
    _epochs = EPOCHS
    _filters = 28
    _kerneln = 3
    _denseunits = 128
    _dropoutfrac = 0.2
    _lr = 0.001
    _inputshape = input_shape
    _x = x_t
    
    if 'a' in scenario:
        pass
    if 'b' in scenario:
        _epochs = EPOCHS*2
    if 'c' in scenario:
        _filters = 28*2
    if 'd' in scenario:
        _kerneln = 6
    if 'e' in scenario:
        _inputshape = input_shape_large
        _x = x_t_large
    if 'f' in scenario:
        _lr=0.002
    if 'g' in scenario:
        _denseunits = 256
    if 'h' in scenario:
        _dropoutfrac=0.3
    if 'i' in scenario:
        _dropoutfrac=0.1
    if 'j' in scenario:
        v_split = 0.2

    
    print('-------------------------------------------------------')
    print(scenario)
    
    mb_y1.setInputShape(_inputshape)
    
    mb_y1.buildModel(filters=_filters,
                     kernel_n=_kerneln,
                     dense_units=_denseunits,
                     dropout_frac=_dropoutfrac,
                     lr=_lr
                     
                    )
    
    t0 = time.time()
    
    mb_y1.fitModel(x=_x, y=y1, epochs=_epochs, validation_split=_vsplit)
    
    t1 = time.time()
    
    time_log_y1[scenario] = t1 - t0
    

### Analyze TimeLog

In [30]:
print('\n'.join([str(x) for x in list(time_log.items())]))

('a', 11.750736236572266)
('b', 20.90665578842163)
('c', 18.07809352874756)
('d', 18.34094548225403)
('e', 45.656052350997925)
('f', 12.67089295387268)
('g', 16.61269187927246)
('h', 12.357426643371582)
('i', 12.280442953109741)
('j', 12.983930110931396)
('k', 12.981396675109863)
('cd', 29.781049489974976)
('fg', 17.16671919822693)
('ec', 78.30956816673279)
('eb', 93.17333030700684)


In [31]:
sum(time_log.values())

413.04993176460266

In [33]:
413-93

320